I was given data set by one startup as technical assignment as a part of interview process.
Its a classification problem.
Training data has 603201 observations
Testing data has 442041 observations /
And owning to some errors due to memory i couldnot complete the assignment.
So I decided to use sample of data and apply the same approach look how the model is performing for the sample data.

# Importing all the required libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize

# Loading the original data sets

In [3]:
UN_train_df = pd.read_csv('C:/Users/smartsyam/Desktop/unitednations/train.csv')
UN_test_df = pd.read_csv('C:/Users/smartsyam/Desktop/unitednations/evaluation.csv')

In [5]:
UN_train_df.head(5)

,storeId,url,additionalAttributes,breadcrumbs,label
0,23,http://www.walmart.com/ip/best-of-toto%3A-pian...,Contributed by=Toto;Format=Paperback;Number of...,books > art music & photography > music,books
1,NaN,NaN,NaN,best buy > computers & tablets > computer card...,rest
2,23,https://www.walmart.com/ip/34441317,Performer=Cult Of Youth;Record Label=Vinyl;1.=...,music on cd or vinyl > rock music on cd or vin...,music
3,26,https://www.overstock.com/Books-Movies-Music-G...,Format=Paperback;Copyright Year=2000;Publisher...,books & media > books > cooking & food books >...,books
4,22,http://www.target.com/p/amore-version-ii/-/A-1...,Store Item Number (DPCI)=244-48-2721;Origin=Ma...,"target > movies, music & books > music > class...",music


Training Data has four features and one target variable. Additionalattributes and breacrumbs are the most information
Bearing variables. Idea was to combine these two and use these as featureset for building our model

In [7]:
UN_test_df.head(5)

,storeId,url,additionalAttributes,breadcrumbs,id
0,NaN,NaN,NaN,baby products > bathing & skin care > washclot...,8589934592
1,NaN,NaN,NaN,online shopping > home & garden > art gal...,8589934593
2,NaN,NaN,NaN,home goods > kitchen & dining > table linens &...,8589934594
3,NaN,NaN,NaN,11320,8589934595
4,NaN,NaN,NaN,tools & home improvement > kitchen & bath fixt...,8589934596


testing data has no label varibale. but one additional id varibale.

As i said earlier idea was to combine the tokens present in two features additionalattribues and breadcrumbs and use them as training fetures. \
First corpus is prepared by combing all the present in these two features and taking most frequents tokens from this
corpus. And then for each observation we look for each token if its a part of this corpus, it will be true if its present
in corpus and false if it is not present.

# taking a sample of 10000 observations from training data.

In [5]:
UN_train_sample = UN_train_df.sample(n=10000)

In [6]:
UN_train_sample.isnull().any()

storeId                  True
url                      True
additionalAttributes     True
breadcrumbs              True
label                   False
dtype: bool

In [7]:
UN_train_sample.isnull().sum()

storeId                 6426
url                     6426
additionalAttributes    6426
breadcrumbs               58
label                      0
dtype: int64

In [8]:
UN_train_sample['label'].value_counts()

rest      6426
books     1607
music     1050
videos     917
Name: label, dtype: int64

In [9]:
UN_train_df['label'].value_counts()

rest      389250
books      95811
music      59996
videos     58144
Name: label, dtype: int64

Comparing the distribution different types of labels in sample. the distirbution is nearly same as orginal data

In [10]:
UN_train_sample.head(5)

,storeId,url,additionalAttributes,breadcrumbs,label
424478,6075,https://www.abebooks.com/servlet/BookDetailsPL...,Book Condition=Fair;Binding=Paperback;Title=Th...,henry james,books
191839,NaN,NaN,NaN,best buy > cell phones > cell phone accessorie...,rest
333091,NaN,NaN,NaN,11320,rest
200921,NaN,NaN,NaN,office > office supplies > office organization...,rest
486321,NaN,NaN,NaN,best buy > computers & tablets > computer card...,rest


In [11]:
UN_train_sample.describe()

,storeId,url,additionalAttributes,breadcrumbs,label
count,3574,3574,3574,9942,10000
unique,11,3570,3560,4574,4
top,23,https://www.walmart.com/ip/Frente-a-Frente/269...,edition=Audio CD,music,rest
freq,844,2,4,266,6426


For further variables has to be in string. so converting features additionalAttributes and breadcrumbs to string

In [ ]:
UN_train_sample['additionalAttributes'] = UN_train_sample['additionalAttributes'].astype(str)
UN_train_sample['breadcrumbs'] = UN_train_sample['breadcrumbs'].astype(str)

Tokenizing these varibales.

In [13]:
UN_train_sample['AddAtt_token'] = UN_train_sample['additionalAttributes'].apply(nltk.word_tokenize)
UN_train_sample['bread_token'] = UN_train_sample['breadcrumbs'].apply(nltk.word_tokenize)

Creating corpus for each observation

In [14]:
UN_train_sample['corpus'] = UN_train_sample['AddAtt_token']+UN_train_sample['bread_token']

Excluding the tokens which are of length 1 as these are mostly symbols and length 2 as i assumed two length words have no meaning also removing NAN words.

In [15]:
UN_train_sample['corpus'] = UN_train_sample['corpus'].apply(lambda x: [item for item in x if len(item) > 2])

In [16]:
UN_train_sample['corpus'] = UN_train_sample['corpus'].apply(lambda x: [item for item in x if item != 'nan'])

Dropping all other features except corpus feature.

In [17]:
UN_train_sample = UN_train_sample.drop(['storeId','url','additionalAttributes','breadcrumbs','AddAtt_token','bread_token'], axis =1)

Creating corpus for this data

In [18]:
total_corpus = []

for x in UN_train_sample.corpus:
    total_corpus.extend(x)
    

total_corpus = nltk.FreqDist(total_corpus)
    
corp_features = list(total_corpus.keys())[0:5000]

I took top 5000 most frequent tokens

making each training observation to a tuple of length 2 with one as a  corpus and other as label

In [19]:
train_final_sample = list(zip(UN_train_sample.corpus, UN_train_sample.label))

Function for searching each token from the observation and then looking out in corpus. and making a dictionary with true if present and false if not present

In [20]:
def find_features(document):
    words = set(document)
    features = {}
    for w in corp_features:
        features[w] = (w in words)

    return features

Creating feature sets of data

In [21]:
sample_train_featuresets = [(find_features(corpus), label) for (corpus, label) in train_final_sample]

diving the sample into training and testing data in the ratio of 0.8 and 0.2

In [22]:
training_sample = sample_train_featuresets[:8000]
testing_sample = sample_train_featuresets[8000:]

Importing few libraries for algorithms

In [23]:
from nltk.classify.scikitlearn import SklearnClassifier

from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC

In [24]:
MNB_classifier = SklearnClassifier(MultinomialNB())
MNB_classifier.train(training_sample)
print("MNB_classifier accuracy percent:", (nltk.classify.accuracy(MNB_classifier, testing_sample))*100)

MNB_classifier accuracy percent: 99.9


In [29]:
BernoulliNB_classifier = SklearnClassifier(BernoulliNB())
BernoulliNB_classifier.train(training_sample)
print("BernoulliNB_classifier accuracy percent:", (nltk.classify.accuracy(BernoulliNB_classifier, testing_sample))*100)

BernoulliNB_classifier accuracy percent: 99.0


In [31]:
LogisticRegression_classifier = SklearnClassifier(LogisticRegression())
LogisticRegression_classifier.train(training_sample)
print("LogisticRegression_classifier accuracy percent:", (nltk.classify.accuracy(LogisticRegression_classifier, testing_sample))*100)

LogisticRegression_classifier accuracy percent: 100.0


In [32]:
SGDClassifier_classifier = SklearnClassifier(SGDClassifier())
SGDClassifier_classifier.train(training_sample)
print("SGDClassifier_classifier accuracy percent:", (nltk.classify.accuracy(SGDClassifier_classifier, testing_sample))*100)

SGDClassifier_classifier accuracy percent: 99.95


In [33]:
LinearSVC_classifier = SklearnClassifier(LinearSVC())
LinearSVC_classifier.train(training_sample)
print("LinearSVC_classifier accuracy percent:", (nltk.classify.accuracy(LinearSVC_classifier, testing_sample))*100)

LinearSVC_classifier accuracy percent: 100.0


So It is observed that even 100 percent accuracy is obtained for testing data. so it can said that this approach is very good 
approach to go forward for this classification problem.
but you can argue that i took only 10000 of total data. But here i believe key step is preparing the corpus which will vary as per the size of total data. also if we look at the data and read through the two features additionalAttributes and breadcrumbs we can easily classify the label by looking at the components of these features. I believe this is what i incorporated in this approach the ability to look into components of these two features.

# please let me know If you have any doubts or suggestions